# 03_telegram_scraper.ipynb
Scrape public Canadian far-right / extremist Telegram channels
→ 100–300+ messages
(For Garon’s PhD – Carleton RA project)

In [1]:
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
import pandas as pd
from pathlib import Path
import asyncio



api_id = 33115930        
api_hash = '10bbd0e83edbf62a53cd0e946d5d7abe'  
phone = '+2348034860145'   

client = TelegramClient('garon_session', api_id, api_hash)

In [2]:
# Public Canadian far-right / adjacent channels 
CHANNELS = [
    "https://t.me/canadianpatriotnetwork",
    "https://t.me/diagonation",
    "https://t.me/plandemicrevolution",
    "https://t.me/canadafirstofficial",
    "https://t.me/therealcanadianpatriot",
    "https://t.me/standupcanada",
    "https://t.me/convoyreports",
    # Add more 
]

# Where to save
TELEGRAM_DIR = Path("data/telegram_raw")
TELEGRAM_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
async def scrape_channel(channel_url):
    await client.start(phone)
    entity = await client.get_entity(channel_url)
    channel_name = entity.username or entity.id
    
    messages = []
    offset_id = 0
    limit = 1000  # max per channel (Telegram limit)
    
    while True:
        history = await client(GetHistoryRequest(
            peer=entity,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=100,
            max_id=0,
            min_id=0,
            hash=0
        ))
        if not history.messages:
            break
        for msg in history.messages:
            messages.append({
                'channel': channel_name,
                'message_id': msg.id,
                'date': msg.date,
                'text': msg.message or "",
                'views': getattr(msg, 'views', 0)
            })
        offset_id = history.messages[-1].id
        if len(history.messages) < 100:
            break
    
    print(f"Scraped {len(messages)} messages from {channel_name}")
    return messages

# RUN THIS CELL — it will ask for your phone + verification code once
all_messages = []
for url in CHANNELS:
    try:
        msgs = await scrape_channel(url)
        all_messages.extend(msgs)
    except Exception as e:
        print(f"Failed {url}: {e}")

df = pd.DataFrame(all_messages)
df.to_csv("telegram_dataset.csv", index=False)
print(f"\nDONE! Saved {len(df)} Telegram messages to telegram_dataset.csv")
df.head(10)

Please enter the code you received:  56261


Signed in successfully as Hesjay; remember to not break the ToS or you will risk an account ban!
Failed https://t.me/canadianpatriotnetwork: No user has "canadianpatriotnetwork" as username
Failed https://t.me/diagonation: No user has "diagonation" as username
Failed https://t.me/plandemicrevolution: No user has "plandemicrevolution" as username
Scraped 1499 messages from CanadaFirstOfficial
Failed https://t.me/therealcanadianpatriot: Nobody is using this username, or the username is unacceptable. If the latter, it must match r"[a-zA-Z][\w\d]{3,30}[a-zA-Z\d]" (caused by ResolveUsernameRequest)


In [ ]:
print(f"Total messages: {len(df)}")
print(f"Date range: {df['date'].min()} → {df['date'].max()}")
print("\nSample messages:")
for text in df['text'].dropna().head(5):
    print("- " + text[:200].replace('\n', ' ') + "...")